<a href="https://colab.research.google.com/github/anilturker/MovingObjectSegmentation/blob/main/MovingObjectSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
gpu_is_available = torch.cuda.is_available()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [4]:
# Dowload movingObjectSegmentation repository from the Github
!git clone https://github.com/anilturker/MovingObjectSegmentation.git

# Get checkpoints and saved weights from drive
!cp -rf /content/drive/MyDrive/results/saved_models /content/MovingObjectSegmentation/

fatal: destination path 'MovingObjectSegmentation' already exists and is not an empty directory.
^C


In [5]:
# Extract drive files to colab directory
import os
import time
import zipfile
start_time = time.time()

currentFrameDrivePath = '/content/drive/MyDrive/dataset/currentFr.zip'
currentFrameFpmDrivePath = '/content/drive/MyDrive/dataset/currentFrFpm.zip'
emptyBgDrivePath = '/content/drive/MyDrive/dataset/emptyBg.zip'
emptyBgFpmDrivePath = '/content/drive/MyDrive/dataset/emptyBgFpm.zip'
recentBgDrivePath = '/content/drive/MyDrive/dataset/recentBg.zip'
recentBgFpmDrivePath = '/content/drive/MyDrive/dataset/recentBgFpm.zip'
fluxTensorDrivePath = '/content/drive/MyDrive/dataset/fluxTensor.zip'

if not os.path.exists('/content/MovingObjectSegmentation/dataset/currentFr'):
  with zipfile.ZipFile(currentFrameDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
if not os.path.exists('/content/MovingObjectSegmentation/dataset/currentFrFpm'):
  with zipfile.ZipFile(currentFrameFpmDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
if not os.path.exists('/content/MovingObjectSegmentation/dataset/emptyBg'):
  with zipfile.ZipFile(emptyBgDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
if not os.path.exists('/content/MovingObjectSegmentation/dataset/emptyBgFpm'):
  with zipfile.ZipFile(emptyBgFpmDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
if not os.path.exists('/content/MovingObjectSegmentation/dataset/recentBg'):
  with zipfile.ZipFile(recentBgDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
if not os.path.exists('/content/MovingObjectSegmentation/dataset/recentBgFpm'):
  with zipfile.ZipFile(recentBgFpmDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
if not os.path.exists('/content/MovingObjectSegmentation/dataset/fluxTensor'):
  with zipfile.ZipFile(fluxTensorDrivePath, 'r') as zip_ref:
    zip_ref.extractall('/content/MovingObjectSegmentation/dataset/')
    
print("Extracting time --- %s seconds ---" % (time.time() - start_time))

Extracting time --- 408.3904206752777 seconds ---


In [ ]:
%%shell
phase=Train
# Install required library
cd /content/MovingObjectSegmentation
pip install -r requirements.txt
# Train
if [ $phase == "Train" ]
then
  echo "Training is started"
  # Run the model
  epoch_count=77
  while true
  do
    python train.py --network 'unetvgg16' --temporal_network 'avfeat_v2' --loss 'jaccard' --opt 'adam' --batch_size 24 --use_selected 200 --model_chk 1 --patch_frame_size 0 --empty_bg 'no' --recent_bg 0 --seg_ch 0 --flux_ch 1 --current_fr 1 --set_number 5 --num_epochs  $epoch_count --inp_size 224 --lr 0.0001
    # Save the results to drive
    cp -rf tb_runs/* /content/drive/MyDrive/results/tb_runs
    cp -rf saved_models/* /content/drive/MyDrive/results/saved_models
    cp -rf *.csv /content/drive/MyDrive/results/
    ((epoch_count=epoch_count+1))
  done
# Inference
else
  echo "Inference is started"
  python inference.py --network 'unetvgg16' --temporal_network 'avfeat' --patch_frame_size 0 --empty_bg 'no' --recent_bg 0 --seg_ch 0 --flux_ch 0 --current_fr 0 --set_number 5
  cp -rf saved_models/* /content/drive/MyDrive/results/saved_models
  cp -rf *.csv /content/drive/MyDrive/results/
fi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Training is started
Model started: MOS-Net_fusion_net_unetvgg16_temporal_net_no_inp_selection_00011_patch_last_frames_0
Initialized TB
=> loading checkpoint '1'
=> loaded checkpoint '1' (epoch 76)
::Train::[77,   100] loss: 5.430, prec: 0.979, recall: 0.969, f_score: 0.974
::Train::[77,   200] loss: 5.176, prec: 0.979, recall: 0.971, f_score: 0.975
::Train::[77,   300] loss: 5.225, prec: 0.979, recall: 0.971, f_score: 0.975
::Train:: Epoch 77 loss: 5.164, prec: 0.979, recall: 0.971, f_score: 0.975, lr : 0.000100, elapsed time: 229.5374083518982
::Test::[77,   100] loss: 46.863, prec: 0.503, recall: 0.780, f_score: 0.612
::Test::[77,   200] loss: 48.106, prec: 0.507, recall: 0.786, f_score: 0.616
::Test::[77,   300] loss: 48.412, prec: 0.487, recall: 0.776, f_score: 0.598
::Test::[77,   400] loss: 49.702, prec: 0.487, recall: 0.775, f_score: 0.598
::Test::[77,   500] loss: 49.018, prec: 0.